# Feature engineering

**Purpose of script:**

Creating new features 

- In: dataframe_plain
- Out: dataframe_extended (with additional feature columns)

In [3]:
import xarray
import pandas as pd
import numpy as np
from tqdm import tqdm

from os import listdir
from os.path import isfile, join

##### Relevant paths

In [4]:
path_dataframe_plain = r"../Data/combined/dataframe_plain/"
mw_path = r"../Data/microwave-rs/mw_interpolated/"
path_elevation =  r"../Data/elevation_data/gimpdem_1km_compressed.tif"
out_path = r"../Data/combined/dataframe_extended/"

## Features

##### Row and column

In [20]:
def add_row_and_col(df):
        # add row and column features:
        df['col'] = df.groupby("x").ngroup() # xshape 2663 
        df['row'] = df.groupby("y").ngroup(ascending=False) # yshape 1462
        return df

##### Date

In [21]:
def add_date(df):
    df['date'] = '2019-07-01'
    df["date"]= pd.to_datetime(df["date"])
    # df['month'] = df["date"].dt.month
    return df

##### Aggregated/pooled values

In [23]:
from typing import Union
from typing import Tuple
from scipy.stats import mode
from scipy.signal import convolve2d

def get_window(image: np.ndarray, window_size: int, center: Tuple[int, int]) -> np.ndarray:
    top = max(center[0] - window_size // 2, 0)
    bottom = min(center[0] + window_size // 2 + 1, image.shape[0])
    left = max(center[1] - window_size // 2, 0)
    right = min(center[1] + window_size // 2 + 1, image.shape[1])
    window = image[top:bottom, left:right]
    return window


# need to fix? : only calculate if the middle value is not nan - else all nan columns around 1 and 0 are going to have a value.

def convolve(image, window_size, convolution_fn: Union['mean', 'min', 'max', 'sum', 'median', 'mode']):
    image = image[0].values
    image[image == -1] = np.nan
    
    if convolution_fn == 'mean':
        kernel = np.ones((window_size, window_size))  # kernel for mean convolution
        result = np.zeros_like(image, dtype=np.float64)
        # Compute the sum and count of non-NaN values in the kernel window
        counts = convolve2d(~np.isnan(image), kernel, mode='same', boundary='fill', fillvalue=0)
        sums = convolve2d(np.nan_to_num(image), kernel, mode='same', boundary='fill', fillvalue=0)
        # Calculate the mean, ignoring NaN values
        result[counts > 0] = sums[counts > 0] / counts[counts > 0]
        # Set the output to NaN where all values in the kernel window are NaN
        result[counts == 0] = np.nan
        return result
    elif convolution_fn == 'max':
        result = np.zeros_like(image, dtype=np.float64)
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                window = get_window(image, window_size, (i, j))
                non_nan_values = window[~np.isnan(window)]
                if len(non_nan_values) == 0:
                    result[i, j] = np.nan
                else:
                    result[i, j] = np.nanmax(non_nan_values)
        return result

    elif convolution_fn == 'min':
        result = np.zeros_like(image, dtype=np.float64)
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                window = get_window(image, window_size, (i, j))
                non_nan_values = window[~np.isnan(window)]
                if len(non_nan_values) == 0:
                    result[i, j] = np.nan
                else:
                    result[i, j] = np.nanmin(non_nan_values)
        return result

    elif convolution_fn == 'sum':
        result = np.zeros_like(image, dtype=np.float64)
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                window = get_window(image, window_size, (i, j))
                non_nan_values = window[~np.isnan(window)]
                if len(non_nan_values) == 0:
                    result[i, j] = np.nan
                else:
                    result[i, j] = np.nansum(non_nan_values)
        return result
        
    elif convolution_fn == 'median':
        result = np.zeros_like(image, dtype=np.float64)
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                window = get_window(image, window_size, (i, j))
                non_nan_values = window[~np.isnan(window)]
                if len(non_nan_values) == 0:
                    result[i, j] = np.nan
                else:
                    result[i, j] = np.nanmedian(non_nan_values)
        return result
    elif convolution_fn == 'mode':
        result = np.zeros_like(image, dtype=np.float64)
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                window = get_window(image, window_size, (i, j))
                non_nan_values = window[~np.isnan(window)]
                if len(non_nan_values) == 0:
                    result[i, j] = np.nan
                else:
                    result[i, j] = mode(non_nan_values)[0]
        return result
    else: 
        print('not available function')
    return

In [24]:
def convolution_to_df(convolution_raster, column = Union['mean', 'min', 'max', 'sum', 'median', 'mode']):
    nrows, ncols = convolution_raster.shape
    # create an array of x and y positions
    x = np.tile(np.arange(ncols), nrows)
    y = np.repeat(np.arange(nrows), ncols)
    # create a DataFrame with x, y, and pixel values as columns
    df = pd.DataFrame({'col': x, 'row': y, column: convolution_raster.flatten()})
    return df 


##### Elevation data

In [25]:
def add_elevation(data):
    df = data.to_dataframe()
    df = df.reset_index()
    df = df[['x', 'y', 'band_data']]
    df.rename({'band_data': 'elevation_data'}, axis=1, inplace=True)
    return df

##### Distance from margin/shore

In [26]:
# tbd
# add if coast column - if at least one na but not all 

## Main:

In [27]:
def get_files(mw_path, path_dataframe_plain):
    # get plain files:
    df_plain_files = [f for f in listdir(path_dataframe_plain) if isfile(join(path_dataframe_plain, f))]
    # microwave files:
    mw_files = [f for f in listdir(mw_path) if isfile(join(mw_path, f))]
    return  mw_files, df_plain_files

In [32]:
def main(mw_files_list, df_plain_files_list, path_elevation, out_path, write = False):
    # get plain files:
    df_plain_files = df_plain_files_list
    # microwave files:
    mw_files = mw_files_list
    # load elevation data:
    data_elevation = xarray.open_dataarray(path_elevation)   

    for df_file in df_plain_files:
        melt_date =  df_file[5:15]
        print(melt_date)
        for mw_file in mw_files:
            if mw_file.startswith(melt_date):
                data_mw = xarray.open_dataarray(mw_path + mw_file)
                df = pd.read_parquet(path_dataframe_plain + df_file)
                # add row and column features:
                df = add_row_and_col(df)
                # get convolution:
                convolution_function = 'mean'
                convolution = convolve(data_mw, 3, convolution_function)
                # convolution to df:
                df_conv = convolution_to_df(convolution, convolution_function)
                # merge convolution:
                df_combined = pd.merge(df, df_conv, how = 'left', on = ['row', 'col'])
                # remove water in mw:
                df_combined = df_combined.loc[df_combined['mw_value'] != -1] # suppress warning?
                # add date:
                df = add_date(df_combined)
                # add elevation data:
                df_elevation = add_elevation(data_elevation)
                # merge elevation data:
                df_with_elevation = pd.merge(df, df_elevation, how = 'left', on = ['y', 'x']) # left smaller mw, right - opt               
                
                # write to parquet:
                if write == True:
                    df_with_elevation.to_parquet(out_path + 'melt_'+ melt_date + '_extended.parquet.gzip', index= False)                    
    return df_with_elevation
                

Main

In [ ]:
#main(mw_path, path_dataframe_plain, path_elevation, out_path)
main(*get_files(mw_path, path_dataframe_plain), path_elevation, out_path)

## Testing

### Testing Lina

In [33]:
df = main(['2019-06-08_mw.tif'], ['melt_2019-06-08.parquet.gzip'], path_elevation, out_path)

2019-06-08


In [34]:
df

,x,y,mw_value,opt_value,col,row,mean,date,elevation_data
0,-636500.0,-662500.0,0.0,-1.0,0,0,0.0,2019-07-01,14.0
1,-635500.0,-662500.0,0.0,-1.0,1,0,0.0,2019-07-01,14.0
2,-634500.0,-662500.0,0.0,-1.0,2,0,0.0,2019-07-01,14.0
3,-633500.0,-662500.0,0.0,-1.0,3,0,0.0,2019-07-01,14.0
4,-632500.0,-662500.0,0.0,-1.0,4,0,0.0,2019-07-01,14.0
...,...,...,...,...,...,...,...,...,...
2278720,58500.0,-3324500.0,0.0,-1.0,695,2662,0.0,2019-07-01,44.0
2278721,59500.0,-3324500.0,0.0,-1.0,696,2662,0.0,2019-07-01,44.0
2278722,60500.0,-3324500.0,0.0,-1.0,697,2662,0.0,2019-07-01,44.0
2278723,61500.0,-3324500.0,0.0,-1.0,698,2662,0.0,2019-07-01,45.0


In [103]:
# # for testing values around the 0-1 change in the data:

# tt = data_mw.values
# # indices = np.where(tt == 1)
# tt[0][74:80, 622:628]